In [ ]:
### Autoreload all modules ###
%load_ext autoreload
%autoreload 2

### import libraries ###
import os
from os.path import join
import sys
from pathlib import Path
import imageio.v2 as imageio
import rasterio as rs
import geopandas as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### Set base path ###
base_path = Path(os.getcwd())
while not (base_path / '.git').exists():
    base_path = base_path.parent
print('Base path: ', base_path)


### Import custom modules ###
sys.path.append(str(base_path / 'src/data/preprocessing'))
sys.path.append(str(base_path / 'src/visualization'))

from preprocess_ukr import preprocess
from filtering import filter_poly, create_diff_masks
from preprocessing_plots import plot_images, plot_diff_mask
from utils import get_files

## Preprocessing

- divides sentinel tiff files into patches
- creates masks from polygons and divides them into patches
- divides cloud masks into patches

In [ ]:
base_path_data =  str(base_path) + '/data/UKR/raw/'

s2_path = base_path_data + 'full/sentinel2'
s1_path = base_path_data + 'full/sentinel1'
save_path = base_path_data + 'pieces'
cloud_path = base_path_data + 'full/clouds'
polygon_files = ['36UYA_36UXA_baseline.geojson', '36UXA_time-dependent.geojson', '36UYA_Spring_time-dependent.geojson', '36UYA_Summer_time-dependent.geojson']
polys_path = [base_path_data + 'full/polygons/' + poly for poly in polygon_files]
filter_by_date = True
width = 224
height = 224
buffer = False


preprocess(
    s2_path,
    s1_path,
    save_path, 
    cloud_path, 
    polys_path,
    width, 
    height, 
    filter_by_date, 
    buffer
)

## Filtering

Filter the pieces according to deforestation change and cloud cover.

In [ ]:
pieces_path = '/scratch/yves/Thesis/Yves-MSc-Thesis/data/UKR/raw/pieces'
save_path = '/scratch/yves/Thesis/Yves-MSc-Thesis/data/UKR/filtered'

filter_poly(pieces_path, save_path, num_deforestation_threshold = 1, cloud_threshold = 0.3)

Filtering post-analysis

In [ ]:
images_path =  base_path + 'data/UKR/filtered/s2'
images = get_files(images_path)
masks = get_files(images_path.replace('s2', 'masks'))


for i in range(0, 10):

    image_path = join(images_path, images[i])
    mask_path = join(images_path.replace('s2', 'masks'), masks[i])
    print(image_path)

    plot_images(image_path, use_mask = False, mask_path = mask_path, use_rgb = True, additional_bands = [4], normalization = 'percentile_stretch', percentile = [1, 99],
                use_s1 = True, titles = ['Sentinel-2 RGB', 'Sentinel-2 NIR', 'Sentinel-1 VV'])

## Difference masking

Creates difference masks from the masks

In [ ]:
image_dir = str(base_path) + '/data/UKR/filtered/s2'
mask_dir = str(base_path) + '/data/UKR/filtered/masks'
save_path = str(base_path) + '/data/UKR/filtered/diff_masks'

create_diff_masks(image_dir, mask_dir, save_path, THRESHOLD_DEF_CHANGE = 0)

Difference masking post-analysis

In [ ]:
images_path =  str(base_path) + '/data/UKR/filtered/diff_masks'
diff_mask_files = get_files(images_path)

for diff_mask in diff_mask_files[0:2]:
    diff_mask_path = join(images_path, diff_mask)

    plot_diff_mask(diff_mask_path, band = 'rgb', s1 = False, s1_band = 1, normalization = 'percentile_stretch', percentile = [2, 98])